On va essayer de clenner les fichier par fichier et voir si y'a moyen d'améliorer ca

In [ ]:
import pandas as pd
import numpy as np
import sys
import glob
import os
import math
import dask.dataframe as dd
import re
import heapq
import numpy as np


In [44]:
parquet_folder = os.path.join(sys.path[0], "data_parquet")
parquet_folder_tracks = os.path.join(sys.path[0], "data_tracks_parquet")

all_files = glob.glob(os.path.join(parquet_folder, "*.parquet"))
all_track_files = glob.glob(os.path.join(parquet_folder_tracks, "*.parquet"))

# Trier les fichiers par le numéro de début de slice
def sort_key(f):
    m = re.search(r"mpd\.slice\.(\d+)-\d+\.parquet", f)
    return int(m.group(1)) if m else float('inf')

all_track_files = sorted(all_track_files, key=sort_key)

On fait un groupage intra fichier

In [ ]:

for i, file in enumerate(all_track_files):
    df = pd.read_parquet(file)
    # groupby local uniquement dans ce fichier
    grouped = (
            df.groupby("track_id").agg({
            "pid": list,             # fusionne la liste de pid
            "track_name": "first",   # garde la première occurrence
            "artist_name": "first"   # idem
        }).reset_index()
    )
    output_file = f"tracks_grouped/grouped_{i:04d}.parquet"
    grouped.to_parquet(output_file, index=False)
    if (i+1) % 100 == 0:
        print(f"Processed {i+1} files")


Processed 0 files
Processed 100 files
Processed 200 files
Processed 300 files
Processed 400 files
Processed 500 files
Processed 600 files
Processed 700 files
Processed 800 files
Processed 900 files


On va trier les fichiers

In [48]:
import pyarrow.parquet as pq
import pyarrow as pa
import os

INPUT_DIR = "tracks_grouped"
TEMP_DIR = "temp_chunks"
os.makedirs(TEMP_DIR, exist_ok=True)

CHUNK_SIZE = 100_000  # nombre de lignes à traiter en mémoire par chunk

chunk_id = 0
for filename in os.listdir(INPUT_DIR):
    if not filename.endswith(".parquet"):
        continue
    table = pq.read_table(os.path.join(INPUT_DIR, filename))
    df = table.to_pandas()

    # Diviser en chunks si fichier très gros
    for start in range(0, len(df), CHUNK_SIZE):
        chunk = df.iloc[start:start + CHUNK_SIZE]
        # Tri en mémoire par track_id
        chunk = chunk.sort_values("track_id")
        # Sauvegarder chunk trié
        chunk_file = os.path.join(TEMP_DIR, f"chunk_{chunk_id:04d}.parquet")
        chunk.to_parquet(chunk_file, index=False)
        chunk_id += 1

print(f"{chunk_id} chunks créés dans {TEMP_DIR}")


1000 chunks créés dans temp_chunks


In [ ]:


TEMP_DIR = "temp_chunks"
OUTPUT_DIR = "track_vectors"
os.makedirs(OUTPUT_DIR, exist_ok=True)

LINES_PER_FILE = 15_000  # lignes par fichier final
BUFFER_SIZE = 15_000       # écrire par lot pour performance

def parquet_row_generator(filename):
    for batch in pq.read_table(filename).to_batches():
        df = batch.to_pandas()
        for _, row in df.iterrows():
            yield row

chunk_files = [os.path.join(TEMP_DIR, f) for f in os.listdir(TEMP_DIR)]
generators = [parquet_row_generator(f) for f in chunk_files]
merged_iter = heapq.merge(*generators, key=lambda r: r["track_id"])

file_idx = 0
lines_in_file = 0
buffer = []

current_id = None
current_list = []

for row in merged_iter:
    tid = row["track_id"]
    lst = row["pid"] 
    art = row["artist_name"]
    name = row["track_name"] 
    if isinstance(lst, np.ndarray):
        lst = lst.tolist()

    if current_id is None:
        current_id = tid
        current_list = lst
        current_artist = art
        current_name = name
    elif tid == current_id:
        current_list.extend(lst)
    else:
        buffer.append({"track_id": current_id, "artist_name": current_artist, "track_name": current_name, "pid": current_list})
        lines_in_file += 1

        current_id = tid
        current_list = lst
        current_artist = art
        current_name = name
    if len(buffer) >= BUFFER_SIZE:
        file_path = os.path.join(OUTPUT_DIR, f"track_vector_{file_idx:04d}.parquet")
        if os.path.exists(file_path):
            # si le fichier existe déjà, créer un nouveau fichier
            file_idx += 1
            lines_in_file = 0
            file_path = os.path.join(OUTPUT_DIR, f"track_vector_{file_idx:04d}.parquet")
        table = pa.Table.from_pandas(pd.DataFrame(buffer))
        pq.write_table(table, file_path)
        buffer = []

    # changer de fichier si LINES_PER_FILE atteint
    if lines_in_file >= LINES_PER_FILE:
        file_idx += 1
        lines_in_file = 0

# écrire la dernière ligne
if current_id is not None:
    buffer.append({"track_id": current_id, "artist_name": current_artist, "track_name": current_name, "pid": current_list})

# écrire ce qui reste
if buffer:
    file_path = os.path.join(OUTPUT_DIR, f"track_vector_{file_idx:04d}.parquet")
    table = pa.Table.from_pandas(pd.DataFrame(buffer))
    pq.write_table(table, file_path)

print(f"Partitionnement terminé dans {OUTPUT_DIR}")


Partitionnement terminé dans track_vectors


In [4]:
parquet_folder_tracks = os.path.join(sys.path[0], "track_vectors")
df_tracks_vectors = dd.read_parquet(parquet_folder_tracks)
df_tracks_vectors = df_tracks_vectors.set_index("track_id", sorted= True)

In [5]:
res = df_tracks_vectors.loc[["7zzpwV2lgKsLke68yFoZdp","7zzmpRP0WkYge45l6LTQ8i"],"pid"].compute()

In [6]:
sim = len(set(res.iloc[0]) & set(res.iloc[1])) / (len(res.iloc[0])**0.5 * len(res.iloc[1])**0.5)

In [7]:
sim

0.0

In [40]:
# Cette fonction évalue la similarité entre deux pistes en fonction des playlists dans lesquelles elles apparaissent.
# De plus on peut lui fournir une liste de pistes ce qui permet immédiatement de résoudre la question 4
def similar_tracks(track_id1, track_id2):
    if isinstance(track_id1, list):
        try:
            subset1 = df_tracks_vectors.loc[track_id1, "pid"].compute()
            subset2 = df_tracks_vectors.loc[track_id2, "pid"].compute()
            list1 = subset1.explode().tolist()
            list2 = subset2.explode().tolist()
        except KeyError:
            print("Le numéro est out of range")
            return None
    else:
        try:
            # Récupérer les colonnes "list_pid"
            subset = df_tracks_vectors.loc[[track_id1, track_id2], "pid"].compute()
            list1,list2 = subset.iloc[0], subset.iloc[1]
        except KeyError:
            print("string")
            return None
    # transformer en sets pour intersection
    set1, set2 = set(list1), set(list2)
    n_common = len(set1 & set2)
    
    if n_common == 0:
        return 0.0

    return n_common / math.sqrt(len(set1) * len(set2))

In [9]:
similar_tracks("7zzpwV2lgKsLke68yFoZdp","7zzpwV2lgKsLke68yFoZdp")

1.0

In [10]:
similar_tracks("7zzpwV2lgKsLke68yFoZdp","08hlaHY7ssHzr2MvBKyKvt")

0.0

Faire une fonction qui prend en entrée le numéro de playlist et ressort la liste de toute les track_id compris dedans

In [12]:
def get_track_ids_for_pid(n: int):
    if not (0 <= n <= 10**6):
        raise ValueError("n doit être entre 0 et 1_000_000 inclus.")
    # Calcul du range
    x = (n // 1000) * 1000
    y = x + 999
    filename = f"mpd.slice.{x}-{y}.parquet"
    # Lecture du fichier parquet
    filepath = os.path.join(sys.path[0], "data_tracks_parquet", filename)
    df = pd.read_parquet(filepath)

    # Filtrer les lignes où pid == n
    filtered = df[df["pid"] == n]

    # Retourner la colonne track_id comme liste
    return filtered["track_id"].tolist()

In [43]:
similar_tracks(get_track_ids_for_pid(987000),get_track_ids_for_pid(50000))

0.05064817454851596